# 1 asyncio: BaseEventLoop

## 目录
- 18.5.1  Base Event Loop
    - 18.5.1.1 运行事件循环
	- 18.5.1.2 调用
	- 18.5.1.3 Delayed calls
	- 18.5.1.4 Futures
	- 18.5.1.5 Tasks
	- 18.5.1.6 Creating connections 
	- 18.5.1.7 Creating listening connections
	- 18.5.1.8 Watch file descriptors
	- 18.5.1.9 Low-level socket operations
	- 18.5.1.10  Resolve host name
	- 18.5.1.11  Connect pipes
	- 18.5.1.12  UNIX signals
	- 18.5.1.13  Executor
	- 18.5.1.14  Error Handling API
	- 18.5.1.15  Debug mode
	- 18.5.1.16  Server
	- 18.5.1.17  Handle
	- 18.5.1.18  Event loop examples
        * 18.5.1.18.1  Hello World with call_soon()
        * 18.5.1.18.2  Display the current date with call_later()
        * 18.5.1.18.3  Watch a file descriptor for read events
        * 18.5.1.18.4  Set signal handlers for SIGINT and SIGTERM

`BaseEventLoop` 是由内核`asyncio`提供多种工具：
    - 注册，执行，延迟处理；
    - 创建服务器和客户端之间的传输通信；
    - 启用子线程和与之相关的传输通信；
    - 将耗费资源的函数响应给线程池。

1. `class asyncio.BaseEventLoop`
    - 具体实现细节的类，继承自 `AbstractEventLoop`类。
    - `BaseEventLoop` 不应该被子类继承; 内部接口不稳定。
    
2. `class asyncio.AbstractEventLoop`
    - `类不是线程安全的。`

### 18.5.1.1 运行事件循环
1. AbstractEventLoop.run_forever()
2. AbstractEventLoop.run_until_complete(future)
3. AbstractEventLoop.is_running()
4. AbstractEventLoop.stop()
5. AbstractEventLoop.is_closed()
6. AbstractEventLoop.close()
7. coroutine AbstractEventLoop.shutdown_asyncgens()

### 18.5.1.2 调用
1. AbstractEventLoop.call_soon(callback, * args)
2. AbstractEventLoop.call_soon_threadsafe(callback, * args)

### 18.5.1.3 延迟调用
1. AbstractEventLoop.call_later(delay, callback, * args)
2. AbstractEventLoop.call_at(when, callback, * args)
3. AbstractEventLoop.time()

### 18.5.1.4 future
1. AbstractEventLoop.create_future()

### 18.5.1.5 任务
1. AbstractEventLoop.create_task(coro)
2. AbstractEventLoop.set_task_factory(factory)
3. AbstractEventLoop.get_task_factory()

### 18.5.1.6 创建连接(待研究)  
1. 
```
coroutine AbstractEventLoop.create_connection(
    protocol_factory, 
    host = None,
    port = None, *,
    ssl = None,
    family = 0,
    proto = 0,
    flags = 0,
    sock = None,
    local_addr = None,
    server_hostname = None)
```
2. 
```
coroutine AbstractEventLoop.create_datagram_endpoint(
    protocol_factory,
    local_addr = None, 
    remote_addr = None, *, 
    family = 0, 
    proto = 0, 
    flags = 0, 
    reuse_address = None, 
    reuse_port = None, 
    allow_broadcast = None,
    sock = None)
```

### 18.5.1.7 创建监听连接

1. ```coroutine AbstractEventLoop.create_server(
            protocol_factory,
            host = None,
            port = None, *,
            family = socket.AF_UNSPEC,
            flags = socket.AI_PASSIVE,
            sock = None,
            backlog = 100,
            ssl = None,
            reuse_address = None,
            reuse_port = None)```
2. ```coroutine AbstractEventLoop.create_unix_server(
            protocol_factory, 
            path, *, 
            ssl=None, 
            sock=None, 
            backlog=100)```
3. ```coroutine BaseEventLoop.connect_accepted_socket(
            protocol_factory, 
            sock, *, 
            ssl=None)```

### 18.5.1.8 观察文件描述符
1. AbstractEventLoop.add_reader(fd, callback, *args)
2. AbstractEventLoop.remove_reader(fd)
3. AbstractEventLoop.add_writer(fd, callback, *args)
4. AbstractEventLoop.remove_writer(fd)

### 18.5.1.9 低级套接字操作
1. coroutine AbstractEventLoop.sock_recv(sock, nbytes)
2. coroutine AbstractEventLoop.sock_sendall(sock, data)
3. coroutine AbstractEventLoop.sock_connect(sock, address)
4. coroutine AbstractEventLoop.sock_accept(sock)

### 18.5.1.10 解析主机名

1. coroutine AbstractEventLoop.getaddrinfo(host, port, *, family=0, type=0, proto=0, flags=0)

### 18.5.1.11 连接管道

1. coroutine AbstractEventLoop.connect_read_pipe(protocol_factory, pipe)
2. coroutine AbstractEventLoop.connect_write_pipe(protocol_factory, pipe)

### 18.5.1.12 UNIX信号

1. AbstractEventLoop.add_signal_handler(signum, callback, *args)
2. AbstractEventLoop.remove_signal_handler(sig)

### 18.5.1.13 Executor 线程池/进程池

1. coroutine AbstractEventLoop.run_in_executor(executor, func, *args)
2. AbstractEventLoop.set_default_executor(executor)

### 18.5.1.14 错误处理
1. AbstractEventLoop.set_exception_handler(handler)
2. AbstractEventLoop.get_exception_handler()
3. AbstractEventLoop.default_exception_handler(context)
4. AbstractEventLoop.call_exception_handler(context)

### 18.5.1.15 调试模式
1. AbstractEventLoop.get_debug()
2. AbstractEventLoop.set_debug(enabled: bool)

### 18.5.1.16 服务器
```
class asyncio.Server
    def close():
    coroutine wait_closed:
    sockets:
```

### 18.5.1.17 处理
```
class asyncio.Handle:
    def cancel():
```

### 18.5.1.18 示例

#### 18.5.1.18.1 Hello World with call_soon()

In [1]:
import asyncio

def hello_world(loop):
    print('Hello World')
    loop.stop()

loop = asyncio.get_event_loop()

# 排定调度计划
loop.call_soon(hello_world, loop)

# Blocking call interrupted by loop.stop()
loop.run_forever()
loop.close()

Hello World


#### 18.5.1.18.2 Display the current date with call_later()

In [2]:
import asyncio
import datetime


def display_date(end_time, loop):
    print(datetime.datetime.now())
    if (loop.time() + 1.0) < end_time:
        loop.call_later(1, display_date, end_time, loop)
    else:
        loop.stop()

loop_t2 = asyncio.get_event_loop()

# Schedule the first call to display_date()
end_time = loop_t2.time() + 5.0
loop_t2.call_soon(display_date, end_time, loop_t2)

# Blocking call interrupted by loop.stop()
loop_t2.run_forever()
# loop_t2.close()

2018-05-27 16:33:37.082851
2018-05-27 16:33:38.083631
2018-05-27 16:33:39.084488
2018-05-27 16:33:40.085343
2018-05-27 16:33:41.085809


### 18.5.1.18.3 Watch a file descriptor for read events

In [3]:
import asyncio
try:
    from socket import socketpair
except ImportError:
    from asyncio.windows_utils import socketpair

# Create a pair of connected file descriptors
rsock, wsock = socketpair()
loop = asyncio.get_event_loop()

def reader():
    data = rsock.recv(100)
    print("Received:", data.decode())
    # We are done: unregister the file descriptor
    loop.remove_reader(rsock)
    # Stop the event loop
    loop.stop()

# Register the file descriptor for read event
loop.add_reader(rsock, reader)

# Simulate the reception of data from the network
loop.call_soon(wsock.send, 'abc'.encode())

# Run the event loop
loop.run_forever()

# We are done, close sockets and the event loop
rsock.close()
wsock.close()
loop.close()

Received: abc


### 18.5.1.18.4 Set signal handlers for SIGINT and SIGTERM

In [1]:
import asyncio
import functools
import os
import signal

def ask_exit(signame):
    print("got signal %s: exit" % signame)
    loop.stop()

loop = asyncio.get_event_loop()
for signame in ('SIGINT', 'SIGTERM'):
    loop.add_signal_handler(
        getattr(signal, signame),
        functools.partial(ask_exit, signame)
    )

print("Event loop running forever, press Ctrl+C to interrupt.")
print("pid %s: send SIGINT or SIGTERM to exit." % os.getpid())
try:
    loop.run_forever()
finally:
    loop.close()

NotImplementedError: 

## APIs

1. `asyncio.Task`

2. `asyncio.async()`
3. `loop.create_task()`
4. `loop.run_until_complte()`

5. `Task.cancel`

6. `asyncio.sleep(DELAY)`  
功能：
    * 非阻塞式的等待。
    * 与`time.sleep(...)`的区别：`time.sleep(...)`除非想阻塞主线程，从而冻结事件循环或整个应用。
    * `yield from asyncio.sleep(DELAY)`

In [1]:
import asyncio

In [ ]:
asyncio.Task